# Federated K-Means
In this section, we attempt to form a federated k-means algorithm to preserve the privacy of the individual labs. 

**Federated learning:**  First introduced by Google, federated learning trains an algorithm across multiple decentralized servers/devices with local data samples. There is **no** data sharing between servers. This is in contrast to the standard way of training a machine learning algorithm where all of the training dataset is uploaded to one server. This technique adrress critical issues relating to data privacy and security 

## Diagram of the federated k-means(FKM) algorithm

In [1]:
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np

In [2]:
dim_reduced_2d = pd.read_csv('data/dim_reduced_2d.tsv', sep="\t")
dim_reduced_3d = pd.read_csv('data/dim_reduced_3d.tsv', sep="\t")


Here, we are just creating variables to hold the data from each study. 

In [3]:
studies_list = ['Horstmann', 'Kjome', 'Maia', 'SteingroverInPrep', 'Premkumar','Wood', 'Worthy', 'Ahn']
for study in studies_list:
    globals()[f'{study}_study'] = dim_reduced_2d[dim_reduced_2d['study'] == study]

### Child K-means class

In [4]:
class child_kmeans(KMeans):
    
    def __init__(self,
                df,
                n_clusters):
        super().__init__(n_clusters=n_clusters)
        self.df = df
        self.update_df, self.base_df = np.split(df, [int(.2*len(df))])

    def update(self):
        pass
        

    def run(self):
        base_run = super().fit(self.base_df) 
        return self.cluster_centers_.tolist(), self.df.shape[0]    
   

### Parent K-means class

In [8]:
class parent_kmeans():
    def __init__(self) -> None:
        self.n_participants = []
        self.cluster_centres = []
        self.federated_cluster_centres = None

    def add(self, cluster_centre, n_participant):
        self.cluster_centres.append(cluster_centre)
        self.n_participants.append(n_participant)

    def weighted_average(self):
        child_weighted_avg = np.average(self.cluster_centres, weights=self.n_participants, axis=0)        
        if self.federated_cluster_centres is None:
            return child_weighted_avg
        return np.mean([self.federated_cluster_centres, child_weighted_avg], axis=0)

    def update_cluster_centre(self):
        self.federated_cluster_centres = self.weighted_average()
        self.n_participants = []
        self.cluster_centres = []
    
    def get_new_centres(self):
        return self.federated_cluster_centres


In [217]:
parent_server = parent_kmeans()

for i in range(1,50):
    for study in studies_list:
        # First retrieving the cluster centres and number of particpants  from a study
        study_cluster_centres, n_particpants = child_kmeans(globals()[f'{study}_study'].iloc[:,2:], n_clusters=3).run()
        # Adding that information to the parent server 
        parent_server.add(cluster_centre=study_cluster_centres, n_participant=n_particpants)

    # Calculating the new federated cluster centres
    parent_server.update_cluster_centre()
    
# Retreving the cluster centres from Federated K-means and normal K-means    
fkm_cluster_centres = parent_server.get_new_centres()
km_clusters_centres = KMeans(n_clusters=3,random_state=42).fit(dim_reduced_2d.iloc[:,2:]).cluster_centers_


In [ ]:
arent server 
        parent_server.add(cluster_centre=study_cluster_centres, n_participant=n_particpants)

    # Calculating the new federated cluster centres
    parent_server.update_cluster_centre()
    
# Retreving the cluster centres from Federated K-means and normal K-means    
fkm_cluster_centres = parent_server.get_new_centres()
km_clusters_centres = KMeans(n_clusters=3,random_state=42).fit(dim_reduced_2d.iloc[:,2:]).cluster_centers_


## Evaluation 

Before evaluating the results, an important consideration should be noted. As with any K-Means algorithm, results may vary with each run of the algorithm as the algorithm's performance is heavily dependent on the initial clusters chosen. Therefore, the federated-k-means algorithms is run 10 times and  the corresponding evaluation metrics are averaged 10 times. The clusters formed in the previous section as the ground truth. Like-kmeans, we must supply the vector of federated k-me

In [ ]:

def calulate_SSE(df, fkm_cluster_centres, km_clusters_centres):
    df["fkm_SSE"] = None
    df["km_SSE"] = None
    for index, subject in df.iterrows():
        subject_dim = subject[["component_1","component_2"]]
        df["fkm_SSE"].loc[index] = min([(np.sum(np.square(subject_dim - cluster))) for cluster in fkm_cluster_centres])
        df["km_SSE"].loc[index] =  min([np.sum(np.square(subject_dim - cluster)) for cluster in km_clusters_centres])
    return df

In [7]:




np.square(subject_cord - cluster)

NameError: name 'subject_cord' is not defined

In [ ]:
subject_cord

component_1   -3.156512
component_2   -2.452422
Name: 628, dtype: object

In [ ]:
cluster

array([-1.08377959, -1.20275671])